Based on https://mitsuba.readthedocs.io/en/latest/src/inverse_rendering/radiance_field_reconstruction.html. To run this section of the tutorial, first run

pip install mitsuba

from the command line.

## Setup
We must import DrJit and Mitsuba and set a variant that supports automatic differentiation. ipywidgets are used to provide an interactive user interface

In [ ]:
import drjit as dr
import mitsuba as mi

import math
import numpy as np

import ipywidgets as widgets
from ipywidgets import interact, fixed

mi.set_variant('cuda_ad_rgb', 'llvm_ad_rgb')

For convenience, we define a helper function to plot a list of images in one row:

In [ ]:
import matplotlib.pyplot as plt
def plot_list(images, title=None):
    fig, axs = plt.subplots(1, len(images), figsize=(18, 3))
    for i in range(len(images)):
        axs[i].imshow(mi.util.convert_to_bitmap(images[i]))
        axs[i].axis('off')
    if title is not None:
        plt.suptitle(title)

Helper function to plot a grid of images

In [ ]:
def plot_grid(images, rows, cols, title=None):
    for i in range(rows):
        plot_list(images[i*cols:(i+1)*cols], title+' (%d)' %i)

## Parameters

We now define a few parameters for the optimization pipeline implemented below. The optimization will start at a low resolution and then upsample the optimized volume parameters every `num_iterations_per_stage` iterations. This will be done a total of `num_stages` times. This coarse-to-fine scheme improves convergence and is a common heuristic used in differentiable rendering.

In [ ]:
# Rendering resolution
render_res = 256

# Number of stages
num_stages = 4

# Number of training iteration per stage
num_iterations_per_stage = 15

# learning rate 
learning_rate = 0.2

# Initial grid resolution
grid_init_res = 16

# Spherical harmonic degree to be use for view-dependent appearance modeling
sh_degree = 2

# Enable ReLU in integrator
use_relu = True

# Number of sensors
# Cameras will be distributed on a sphere around the center of the scene
num_cols = 7
num_rows = 1

sensor_count = num_rows * num_cols

## Creating multiple sensors

As done in many of the other tutorials, we instantiate a couple of sensors to render our synthetic scene from different viewpoints. Here the cameras are placed in a circle around the `[0.5, 0.5, 0.5]` point which is going to be the center of our voxel grids.

Define a helper class to more easily enable interactive movement of the camera

In [ ]:
class Camera:
    #defaults
    eyePoint = [0,0,0]
    aimPoint = [0,0,1]
    upVector = [0,1,0]
    fov = 45

    latitude = 0
    longitude = 0

    radius = 1

    def __init__(self, eyePoint = [0,0,0], aimPoint = [0,0,1], upVector = [0,1,0], fov = 45):
        self.eyePoint = eyePoint
        self.aimPoint = aimPoint
        self.upVector = upVector
        self.fov = fov

    #define the position of camera relative to aimPoint in spherical coordinates
    def placeCamera(self, latitude, longitude, radius):
        self.latitude = latitude
        self.longitude = longitude
        self.radius = radius

        self.eyePoint[0] = self.aimPoint[0]+radius*math.sin(longitude * math.pi/180)*math.cos(latitude * math.pi/180)
        self.eyePoint[1] = self.aimPoint[1]+radius*math.sin(latitude * math.pi/180)
        self.eyePoint[2] = self.aimPoint[2]+radius*math.cos(longitude * math.pi/180)*math.cos(latitude * math.pi/180)

Helper function to load a Mitsuba sensor with the parameters of a Camera object

In [ ]:
def loadSensor(camera):
    return mi.load_dict({
        'type': 'perspective',
        'fov': camera.fov,
        'to_world': mi.ScalarTransform4f.look_at(target = camera.aimPoint, origin = camera.eyePoint, up = camera.upVector),
        'film': {
            'type': 'hdrfilm',
            'width': render_res,
            'height': render_res,
            'filter': {'type': 'box'},
            'pixel_format': 'rgba'
        }
    })

In [ ]:
scene_ref = mi.load_file('scenes/lego/scene.xml')

sensors = []
cameras = []
def placeMultipleCameras(rows, cols):
    for i in range(rows):
        for j in range(cols):
            longitude = 360.0 / cols * j
            latitude = 90/rows * i

            camera = Camera([0,0,0], [.5,.5,.5], [0,1,0], 45)

            camera.placeCamera(latitude, longitude, 1.3)

            sensor = loadSensor(camera)
            cameras.append(camera)
            sensors.append(sensor)

placeMultipleCameras(num_rows, num_cols)

This helper function creates a GUI to edit any of the cameras in the scene

In [ ]:
def interactive_render(index, scene, latitude, longitude, radius, fov):
    camera = Camera([0,0,0], [.5,.5,.5], [0,1,0], fov)
    camera.placeCamera(latitude,longitude,radius)

    sensor = loadSensor(camera)

    cameras[index] = camera
    sensors[index] = loadSensor(camera)

    image = mi.render(scene, sensor=sensors[index], spp=64)
    tone_mapped = image.numpy() ** (.5)
    tone_mapped[tone_mapped > 1] = 1

    plt.axis("off")
    plt.imshow(tone_mapped)

Use the index dropdown to select a camera to edit

In [ ]:
cam_index = 0

def select_camera(index, scene):
    cam_index = index
    interact(interactive_render,
        index = fixed(cam_index),
        scene = fixed(scene),
        latitude = widgets.FloatSlider(min = -89, max = 89, value = cameras[cam_index].latitude),
        longitude = widgets.FloatSlider(min = 0, max = 360, value = cameras[cam_index].longitude),
        radius = widgets.FloatSlider(min = 0, max = 10, value = cameras[cam_index].radius),
        fov = widgets.FloatSlider(min = 0, max = 90, value = cameras[cam_index].fov)
)

interact(select_camera, index = widgets.Dropdown(options = range(sensor_count)), scene = fixed(scene_ref))

Run this cell to show all views at once

In [ ]:
ref_images = [mi.render(scene_ref, sensor=sensors[i], spp=64) for i in range(sensor_count)]

plot_grid(ref_images, num_rows, num_cols, "References images")

## NeRF-like PRB integrator

Unlike the other tutorials, this pipeline doesn't use any of the conventional physically-based rendering algorithms such as path-tracing. Instead, it implements a differentiable integrator for emissive volumes that directly uses a density and SH coefficient grid.

We define an integrator `RadianceFieldPRB` that inherits from `mi.ad.common.RBIntegrator`. The `RBIntegrator` class is meant to be used to implement differentiable integrators 
that rely on tracing light paths to estimate derivatives (in contrast to purely using automatic differentiation). In the Mitsuba code base, this integrator base class is primarily used to implement various versions of [path replay backpropagation][1]. 

In the following implementation, we use this interface to implement a differentiable rendering method for purely emissive volumes (we don't consider scattering or indirect effects). We will use the path replay algorithm to implement a ray marching routine that does not need to allocate any large temporary buffers to differentiate the rendering process. This means that the memory use of this implementation will only depend on the size of the parameter grids, and not on the number of rays being evaluated at once.

We implement the functions `__init__`, `eval_emission` `sample`, `traverse` and `parameters_changed`.

In `__init__` we initialize a bounding box for our volume as well as 3D textures storing the density (`sigmat`) and SH coefficients (`sh_coeffs`). The number of channels for the SH coefficients depends on the chosen `sh_degree` (default: `2`). 

The `traverse` function simply returns the differentiable parameters and `parameters_changed` updates the 3D textures in case the differentiable parameters were updated (e.g., by a gradient step). The update of the 3D textures is necessary for Mitsuba variants where hardware-accelerated texture interpolation is used (i.e., `cuda_ad_rgb`). By invoking `texture.set_tensor(texture.tensor())`, we force Dr.Jit to update the underlying hardware texture.

The main ray marching implementation is inside the `sample` function. This function returns the radiance along a single input ray. We first check if the ray intersects the volume's bounding box. We then use an `mi.Loop` to perform the ray marching routine. Inside the loop, we evaluate the density and spherical harmonics coefficients at the current point `p`. We accumulate radiance in a variable `L` and use `β` to store the current throughput. The directionally varying emission is evaluated using the `eval_emission` helper function. 

The `sample` function is written such that it can both be used for the primal computation (`primal = True`) and to accumulate gradients in reverse mode (`primal = False`). 

When computing the parameter gradients, we pass the gradient of the objective function (`δL`) and the output of the primal computation (`state_in`). Instead of accumulating radiance in the variable `L`, we initialize `L` from the primal output and then iteratively *subtract* emitted radiance to reconstruct gradient values. The `dr.backward_from` call backpropagates gradients all the way to the two 3D textures.

In reverse mode, a significant part of the computation inside the loop is evaluated with gradients enabled. The `dr.resume_grad` scope enables gradients selectively inside the `with` block. The algorithm is designed to not build any AD graph across loop iterations.

For a more detailed explanation of path replay, please see the [paper][1].


[1]: http://rgl.epfl.ch/publications/Vicini2021PathReplay

In [ ]:
class RadianceFieldPRB(mi.ad.common.RBIntegrator):
    def __init__(self, props=mi.Properties()):
        super().__init__(props)
        self.bbox = mi.ScalarBoundingBox3f([0.0, 0.0, 0.0], [1.0, 1.0, 1.0])
        self.use_relu = use_relu
        self.grid_res = grid_init_res
        # Initialize the 3D texture for the density and SH coefficients
        res = self.grid_res
        self.sigmat = mi.Texture3f(dr.full(mi.TensorXf, 0.01, shape=(res, res, res, 1)))
        self.sh_coeffs = mi.Texture3f(dr.full(mi.TensorXf, 0.1, shape=(res, res, res, 3 * (sh_degree + 1) ** 2)))

    def eval_emission(self, pos, direction): 
        spec = mi.Spectrum(0)
        sh_dir_coef = dr.sh_eval(direction, sh_degree)
        sh_coeffs = self.sh_coeffs.eval(pos)
        for i, sh in enumerate(sh_dir_coef):
            spec += sh * mi.Spectrum(sh_coeffs[3 * i:3 * (i + 1)])
        return dr.clip(spec, 0.0, 1.0)

    def sample(self, mode, scene, sampler,
               ray, δL, state_in, active, **kwargs):
        primal = mode == dr.ADMode.Primal
        
        ray = mi.Ray3f(ray)
        hit, mint, maxt = self.bbox.ray_intersect(ray)
        
        active = mi.Bool(active)
        active &= hit  # ignore rays that miss the bbox
        if not primal:  # if the gradient is zero, stop early
            active &= dr.any(dr.neq(δL, 0))

        step_size = mi.Float(1.0 / self.grid_res)
        t = mi.Float(mint) + sampler.next_1d(active) * step_size
        L = mi.Spectrum(0.0 if primal else state_in)
        δL = mi.Spectrum(δL if δL is not None else 0)
        β = mi.Spectrum(1.0) # throughput
        
        loop = mi.Loop(name=f"PRB ({mode.name})",
                       state=lambda: (sampler, ray, L, t, δL, β, active))
        while loop(active):
            p = ray(t)
            with dr.resume_grad(when=not primal):
                sigmat = self.sigmat.eval(p)[0]
                if self.use_relu:
                    sigmat = dr.maximum(sigmat, 0.0)
                tr = dr.exp(-sigmat * step_size)
                # Evaluate the directionally varying emission (weighted by transmittance)
                Le = β * (1.0 - tr) * self.eval_emission(p, ray.d) 
                if not primal:
                    dr.backward_from(δL * (L * tr / dr.detach(tr) + Le))
            β *= tr
            L = L + Le if primal else L - Le
            t += step_size
            active &= (t < maxt) & dr.any(dr.neq(β, 0.0))

        return L if primal else δL, mi.Bool(True), L

    def traverse(self, callback):
        callback.put_parameter("sigmat", self.sigmat.tensor(), mi.ParamFlags.Differentiable)
        callback.put_parameter('sh_coeffs', self.sh_coeffs.tensor(), mi.ParamFlags.Differentiable)

    def parameters_changed(self, keys):
        self.sigmat.set_tensor(self.sigmat.tensor())
        self.sh_coeffs.set_tensor(self.sh_coeffs.tensor())
        self.grid_res = self.sigmat.shape[0]

mi.register_integrator("rf_prb", lambda props: RadianceFieldPRB(props))

## Setting up the optimization scene

Here we set up our simple optimization scene. It is only composed of a constant area light and a `RadianceFieldPRB` integrator. No geometry or volume instance is needed since the integrator itself already contains the feature voxel grid.

As shown in the rendered initial state, the scene appears empty at first as the density grid was initialized with a very low density value.

In [ ]:
scene = mi.load_dict({
    'type': 'scene', 
    'integrator': {
        'type': 'rf_prb'
    }, 
    'emitter': {
        'type': 'constant'
    }
})
integrator = scene.integrator()

# Render initial state
init_images = [mi.render(scene, sensor=sensors[i], spp=128) for i in range(sensor_count)]

plot_grid(init_images, num_rows, num_cols, 'Initialization')

## Optimization

We use an `Adam` optimizer in this pipeline. The constructor of the optimizer takes the learning rate as well as a dictionary of optimized variables. In this tutorial, we want to optimize the density and spherical harmonics coefficients grids. We call `params.update(opt)` to ensure that the integrator is notified that some parameters now have gradients enabled.

In [ ]:
params = mi.traverse(integrator)
opt = mi.ad.Adam(lr=learning_rate, params={'sigmat': params['sigmat'], 'sh_coeffs': params['sh_coeffs']})
params.update(opt);

Finally comes the main optimization loop of the pipeline. As in previous tutorials, at every iteration we render the scene from the different viewpoints and back-propagate the gradients through an L1 loss.

For convenience we store intermediate renderings at the end of every stage to further inspect the optimization progress.

Moreover, as stated earlier in this tutorial, we up-sample the feature voxel grids by a factor of two at the end of every stage. This can easily be achieved using the [<code>dr.upsample()</code>][1] routine.

[1]: https://drjit.readthedocs.io/en/latest/src/api_reference.html#upsample

In [ ]:
losses = []
intermediate_images = []

for stage in range(num_stages):
    print(f"Stage {stage+1:02d}, feature voxel grids resolution -> {opt['sigmat'].shape[0]}")
    
    for it in range(num_iterations_per_stage):
        total_loss = 0.0
        images = []
        for sensor_idx in range(sensor_count):
            img = mi.render(scene, params, sensor=sensors[sensor_idx], spp=1, seed=it)
            loss = dr.mean(dr.abs(img - ref_images[sensor_idx]))
            dr.backward(loss)
            total_loss += loss[0]
            
            # Store images at the end of every stage
            if it == num_iterations_per_stage - 1:
                dr.eval(img)
                images.append(img)
            
        losses.append(total_loss)
        opt.step()

        if not integrator.use_relu:
            opt['sigmat'] = dr.maximum(opt['sigmat'], 0.0)
            
        params.update(opt)
        print(f"  --> iteration {it+1:02d}: error={total_loss:6f}", end='\r')

    intermediate_images.append(images)
    
    # Upsample the 3D textures at every stage
    if stage < num_stages - 1: 
        new_res = 2 * opt['sigmat'].shape[0]
        new_shape = [new_res, new_res, new_res]
        opt['sigmat']   = dr.upsample(opt['sigmat'],   new_shape)
        opt['sh_coeffs'] = dr.upsample(opt['sh_coeffs'], new_shape)
        params.update(opt)

print('')
print('Done')

## Results

We render the final images at higher SPP and display the results at every stages as well as the reference images.

In [ ]:
final_images = [mi.render(scene, sensor=sensors[i], spp=128) for i in range(sensor_count)]
for stage, inter in enumerate(intermediate_images):
    plot_grid(inter, num_rows, num_cols, f'Stage {stage}')

plot_grid(final_images, num_rows, num_cols, 'Final')

plot_grid(ref_images, num_rows, num_cols, 'Reference')

We can also take a closer look at one of the view point:

In [ ]:
fig, axs = plt.subplots(1, 2, figsize=(10, 4))
axs[0].imshow(mi.util.convert_to_bitmap(final_images[1]))
axs[0].set_title('Reconstructed')
axs[0].axis('off')
axs[1].imshow(mi.util.convert_to_bitmap(ref_images[1]))
axs[1].set_title('Reference')
axs[1].axis('off');

When working with optimization pipelines, it is always very informative to take a look at the graph of objective function values.

In [ ]:
plt.plot(losses)
plt.xlabel('Iterations')
plt.ylabel('Loss')

[[-1, 0, 0, 0.5],
 [0, 1, 0, 0.5],
 [0, 0, -1, 1.8],
 [0, 0, 0, 1]]
[[-0.62349, 0, -0.781831, 1.51638],
 [0, 1, 0, 0.5],
 [0.781831, 0, -0.62349, 1.31054],
 [0, 0, 0, 1]]
[[0.222521, 0, -0.974928, 1.76741],
 [0, 1, 0, 0.5],
 [0.974928, 0, 0.222521, 0.210723],
 [0, 0, 0, 1]]
[[0.900969, 0, -0.433884, 1.06405],
 [0, 1, 0, 0.5],
 [0.433884, 0, 0.900969, -0.67126],
 [0, 0, 0, 1]]
[[0.900969, 0, 0.433884, -0.0640486],
 [0, 1, 0, 0.5],
 [-0.433884, 0, 0.900969, -0.67126],
 [0, 0, 0, 1]]
[[0.222521, 0, 0.974928, -0.767406],
 [0, 1, 0, 0.5],
 [-0.974928, 0, 0.222521, 0.210723],
 [0, 0, 0, 1]]
[[-0.62349, 0, 0.781831, -0.516381],
 [0, 1, 0, 0.5],
 [-0.781831, 0, -0.62349, 1.31054],
 [0, 0, 0, 1]]

Interactively view the final radiance field. Note the severe reconstruction errors in interpolated views. A much larger number of input views (100 is typical for 360 degree camera movement) is required for higher reconstruction quality. Try editing the training parameters above and running this notebook again

In [ ]:
interact(select_camera, index = widgets.Dropdown(options = range(sensor_count)), scene = fixed(scene))

## See also

- [<code>mitsuba.ad.integrators.RBIntegrator</code>](https://mitsuba.readthedocs.io/en/latest/src/api_reference.html#mitsuba.ad.integrators.RBIntegrator)
- [<code>mitsuba.TensorXf</code>](https://mitsuba.readthedocs.io/en/latest/src/api_reference.html#mitsuba.TensorXf)
- [<code>mitsuba.Texture3f</code>](https://mitsuba.readthedocs.io/en/latest/src/api_reference.html#mitsuba.Texture3f)
- [<code>dr.sh_eval</code>](https://drjit.readthedocs.io/en/latest/src/api_reference.html#sh_eval)
- [<code>dr.upsample</code>](https://drjit.readthedocs.io/en/latest/src/api_reference.html#upsample)